In [ ]:
#Again we use the "!" command to specify we want to execute commands via the terminal, outside python
#rootls is an example of a root command line tool: 
#https://root.cern/manual/storing_root_objects/#root-command-line-tools
#Allows to run common operations from the linux/mac terminal, without running ROOT or python directly.
! rootls ../Lecture1/tree2.root

In [ ]:
#To do the same in python we would have to type:
from ROOT import TFile
myFile = TFile.Open("../Lecture1/tree2.root")
#Note the output is a bit more detailed though
myFile.ls()

In [ ]:
#"make" is a standard command to build an executable, which you can run. An executable can be thought of as a 
#translation of your computer code into something the computer can understand. When using python this happens in
#real time, behind the scenes.
#The "!" tells Jupyter to run the command in the underlying terminal.
! make 
#You can see the commands "make" actually runs printed below.
#Lots of options passed to g++ (don't worry about these) - the important part is it creates library files (*.d,*.o)
#and eventually an executable which you run ("myROOTCommand")

In [ ]:
#Run the command in the terminal, and we see what it does.
! ./myROOTCommand ../Lecture1/tree2.root t2

In [ ]:
#We can also make a python shortcut command!
#Note in linux you can skip the "python3" part (needed in Jupyter on mybinder!)
! python3 ./myROOTCommand.py ../Lecture1/tree2.root t2

In [ ]:
#Task 1
#Make your own python shortcut command which draws an arbitrary variable from the TTree in tree2.root

In [ ]:
#Now lets use uproot - note there is no c++ version! Python only.
import uproot
#Load the TTree from the ATLAS open data (note the miniTree variable is NOT of type TTree - we are now
#in the uproot ecosystem and not in the ROOT ecosystem
miniTreeZmumu = uproot.open("https://atlas-opendata.web.cern.ch/Legacy13TeV/2lep/MC/mc_364100.Zmumu_PTV0_70_CVetoBVeto.2lep.root")["mini"]
#Lets print the variables in the TTree
miniTreeZmumu.keys()
#note these variables are defined at https://atlas-opendata.web.cern.ch/docs/datasets/dataset13

In [ ]:
#choose which variables we want to use
#entry_stop restricts the number of entries to use (just done to make example run faster...)
#library set to 'pd' gives us a pandas dataframe
df_Zmumu = miniTreeZmumu.arrays(["lep_n","jet_n"],library='pd',entry_stop=1000)
df_Zmumu

In [ ]:
#In my notebook, had to run this cell twice to get the histograms to display...
df_Zmumu.hist("jet_n")

In [ ]:
miniTreeTTbar = uproot.open("https://atlas-opendata.web.cern.ch/Legacy13TeV/2lep/MC/mc_410000.ttbar_lep.2lep.root")["mini"]
df_ttbar = miniTreeTTbar.arrays(["jet_n","jet_pt"],library='pd',entry_stop=1000)
#jet_pt is an "awkward array" (i.e of variable size), stored as a series
#so we have to use this procedure to flatten it to a 1-D array that 
#the hist function would use.
#First we extract the jet_pt as an array of lists (one list per event)
jet_pt = (df_ttbar['jet_pt']).to_numpy()
#Then we flatten this into one array of all jet pt
import numpy as np
flat_jet_pt = np.concatenate(jet_pt)
import matplotlib.pylab as plt
#Make histogram using pylab hist function
jet_ptHist = plt.hist(flat_jet_pt,bins=10,range=[0,100000])
#in python prompt you need to run plt.show() at this step, not needed in Jupyter
plt.title("Jet Pt")
plt.xlabel("Jet Pt (MeV)")
plt.ylabel("Number of Jets")
#Other variable types can be histogrammed directly using the pandas array
df_ttbar.hist("jet_n",bins=10)
plt.title("Number of Jets")
plt.xlabel("Number of Jets")
plt.ylabel("Number of Events")

In [ ]:
#Task 2
#Try making some histograms of other quantities in the open data.
#Can you reproduce plots we saw in earlier Lectures with .../Lecture1/tree2.root using uproot and matplotlib?

In [ ]:
#redefine ttbar array with same variables as Zmumu array
df_ttbar = miniTreeTTbar.arrays(["lep_n","jet_n"],library='pd',entry_stop=1000)
df_Zmumu=df_Zmumu.assign(signal=1)
df_ttbar=df_ttbar.assign(signal=0)
#Create a list of the two pandas data frames
df_list = [df_Zmumu,df_ttbar]
#Convert this new list to one pandas data frame
import pandas
df_all = pandas.concat(df_list)
#In case we only have integers, make sure they are treated as floats (ML software usually requires variables to be floats)
df_all = df_all.astype('float64')
df_all

In [ ]:
df_all.hist("signal")

In [ ]:
dataset = df_all.values
X = dataset[:,0:2]
Y = dataset[:,2]

In [ ]:
#ML prefers to have variables distributed in a specific way
#Here we require them to have mean of zero and standard deviation of 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
import keras as ks
keras_model = ks.Sequential()
keras_model.add(ks.layers.Dense(12,activation='relu'))
keras_model.add(ks.layers.Dense(8, activation='relu'))
keras_model.add(ks.layers.Dense(1, activation='sigmoid'))

In [ ]:
keras_model.compile(optimizer='adam', loss='binary_crossentropy')
keras_model.fit(X,Y,epochs=10,batch_size=32)

In [ ]:
import numpy as np

predictions = keras_model.predict(X)

#Change our predicted probability to a binary 0 or 1
def final_predictions(predictions):
    finalPredictions = np.zeros((len(predictions),))
    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            finalPredictions[i] = 1
        else:
            finalPredictions[i] = 0

    return finalPredictions

#Count how often our predictin matches the true signal value
def count_correct_predictions(predictions, Y):
    nCorrect = 0
    for i in range(len(predictions)-1):
        if predictions[i] == Y[i]:
            nCorrect += 1
    return nCorrect

In [ ]:
finalPredictions = final_predictions(predictions)
print("Number of correct predictions are ", count_correct_predictions(finalPredictions, Y))

In [ ]:
#Build the same model using Google DeepMinds Sonnet software
import sonnet as snt
import tensorflow as tf

def build_model():
  model = snt.Sequential([
      snt.Linear(12),
      tf.nn.relu,
      snt.Linear(8),
      tf.nn.relu,
      snt.Linear(1),
      tf.nn.sigmoid
  ])
  return model

In [ ]:
#Build our model
sonnet_model = build_model()

#Define a function to train our model
def train_model(model, X, Y, epochs=10, batch_size=32, learning_rate=0.001):
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = tf.keras.losses.BinaryCrossentropy()

    for epoch in range(epochs):        
        for i in range(0, len(X), batch_size):
            batch_X = X[i:i+batch_size]
            batch_y = Y[i:i+batch_size]

            with tf.GradientTape() as tape:
                predictions = model(batch_X)
                loss = loss_fn(batch_y, predictions)

            gradients = tape.gradient(loss, model.trainable_variables)

            optimizer.apply_gradients(zip(gradients, model.trainable_variables))


In [ ]:
#Train this sonnet model and check its performance

train_model(sonnet_model, X,Y)
predictions = sonnet_model(X)
print("sonnet predictions are: ", predictions)

finalPredictions = final_predictions(predictions)
print("Number of correct predictions are ", count_correct_predictions(finalPredictions, Y))